In [1]:
import os
current_directory = str(os.getcwd())

In [8]:
import numpy as np
from sympy import *
from fractions import Fraction
import pickle
from scipy.interpolate import griddata
import pandas as pd


#converting from 2nd unit to 1st
pc_kpc = 1e3  # number of pc in one kpc
cm_km = 1e5  # number of cm in one km
s_day = 24*3600  # number of seconds in one day
s_min = 60  # number of seconds in one hour
s_hr = 3600  # number of seconds in one hour
cm_Rsun = 6.957e10  # solar radius in cm
g_Msun = 1.989e33  # solar mass in g
cgs_G = 6.674e-8
cms_c = 2.998e10
g_mH = 1.6736e-24
g_me = 9.10938e-28
cgs_h = 6.626e-27
deg_rad = 180e0/np.pi
arcmin_deg = 60e0
arcsec_deg = 3600e0
cm_kpc = 3.086e+21  # number of centimeters in one parsec
cm_pc = cm_kpc/1e+3
s_Myr = 1e+6*(365*24*60*60)  # megayears to seconds

#REQUIRED FUNCTIONS
###########################################################################################################################################
#importing data from csv file to np array
def file_reader(list_of_file_names):
    dataframe_list=[]
    for i in list_of_file_names:
        df = pd.read_csv(current_directory+r'\{}.dat'.format(i),delimiter=',')
        dataframe_list.append(df)
    return dataframe_list

#extrapolation
def interpolation(list1,list2,standard):
    interpolated_data = griddata(list1, list2, standard, method='linear', fill_value=nan, rescale=False)
    return interpolated_data
###########################################################################################################################################
os.chdir(current_directory+'\M31_data')
raw_data = pd.read_csv('combined_data_m31.csv', skiprows=1)
os.chdir(current_directory)

In [ ]:
#to obtain radius data from every df
distance_m31= 0.78 #Mpc
distances_Mpc=[0.785,0.785,0.785,0.785,0.785,0.785,0.780,0.780,0.785,0.785]
r_col_names=['r kpc','r arcmin','r kpc','r arcmin','r arcmin','r kpc','r kpc','r arcmin','r arcsec','r arcsec']
radius_conv_corr=[distance_m31/distances_Mpc[0],(1/(arcmin_deg*deg_rad))*(distance_m31*1000),distance_m31/distances_Mpc[2],
                  (1/(arcmin_deg*deg_rad))*(distance_m31*1000),(1/(arcmin_deg*deg_rad))*(distance_m31*1000),distance_m31/distances_Mpc[5],
                  distance_m31/distances_Mpc[6],(1/(arcmin_deg*deg_rad))*(distance_m31*1000),(1/(60*arcmin_deg*deg_rad))*(distance_m31*1000),
                  (1/(60*arcmin_deg*deg_rad))*(distance_m31*1000)] 

#obtain arrays of quantities
col_names=['sigma_tot','sigma_HI','sigma_HI','vcirc kms','error kms','vcirc kms',
           'sigma_sfr','molfrac','vdisp kms no_warp','vdisp kms warp']
conv_factors=[(g_Msun/(cm_pc**2) ), g_Msun/(cm_pc**2), g_Msun/(cm_pc**2), 1,1,1,
              g_Msun/((s_Myr*10**(3))*(cm_pc**2)),1, (3**0.5),(3**0.5)] #last term to make 3D vdisp
